Project: Building a database for crime reports.

This project is to better understand how to use PostgreSQL inorder to build databases. 

In [92]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

ProgrammingError: database "crime_db" already exists


In [102]:
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")


In [94]:
import csv
with open("boston.csv", "r") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [95]:
import pandas as pd

def get_col_set(csv_filename, col_index):
    reader = pd.read_csv(csv_filename) 
    unique = set(reader.iloc[:,col_index].unique())
    return unique
    
    

In [96]:
values_per_col_index = {}
for i in range(7):
    values_per_col_index[i] = len(get_col_set('boston.csv', i))
print(values_per_col_index)

{0: 298329, 1: 219, 2: 239, 3: 1177, 4: 7, 5: 18177, 6: 18177}


In [83]:
print(col_headers)
max_length = 0
for data in get_col_set("boston.csv", 6):
    if(data>max_length):
        max_length = data
print(max_length)
    

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
-71.17867378


In [100]:
print(col_headers)
print(first_row)
cur.execute("ROLLBACK")

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [103]:
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code SMALLINT,
        offense VARCHAR(64),
        date DATE,
        day VARCHAR(10),
        latitude NUMERIC(10,8),
        longitude NUMERIC(11,8))""")

In [104]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [105]:
cur.execute("REVOKE ALL ON schema public FROM public")
cur.execute("REVOKE ALL ON database crime_db FROM public")

In [106]:
cur.execute("CREATE GROUP readonly NOLOGIN")
cur.execute("CREATE GROUP readwrite NOLOGIN")

In [107]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly, readwrite")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly, readwrite")

In [108]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite")

In [109]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
cur.execute("GRANT readonly TO data_analyst")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")
cur.execute("GRANT readwrite TO data_scientist")

In [111]:
cur.execute("""SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';""")

In [112]:
result = cur.fetchall()

In [113]:
print(result)

[('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]
